In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

yu.flag_fast=False
yu.slName='20240218_ETMC'

In [ ]:
ensembles=['cA211.530.24','cA2.09.48']

ens2data={}
for ens in ensembles:
    path={
        'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_c/data/cA211.530.24_NST_a_daint.h5',
        'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_c/data/cA2.09.48_NST_c_booster.h5',
        }[ens]
    ens2data[ens]=yu.load(path)
    print('Ncfg='+str(yu.deepKey(ens2data[ens]['2pt'],2).shape[0]))
    print()

lat=None
data=None
Ncfg=None
Tpack=None
    
def updateEns(ens):
    global lat,data,Ncfg,Tpack
    lat=yu.LatticeEnsemble(ens)
    data=ens2data[ens]
    Ncfg=yu.deepKey(data['2pt'],2).shape[0]
    Tpack=yu.TPack[ens]
updateEns(ensembles[0])

In [ ]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'remove_pi0VEV':True, 
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    res=np.zeros([Ncfg,Tpack],dtype=complex)
    if opa+'_'+opb in data['2pt']:
        t=data['2pt'][opa+'_'+opb]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt']:
        t=data['2pt'][opb+'_'+opa]; 
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0))
    return res
def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    res=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)
    if opa+'_'+opb in data['3pt']:
        t=data['3pt'][opa+'_'+opb][insert]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['3pt']:
        t=data['3pt'][opb+'_'+opa][insert]; 
        diags_cc={'B3pt','W3pt','Z3pt','T-j', 'T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f'}
        if opa+'_'+opb not in data['3pt'] or 'NJN-pi0f' not in data['3pt'][opa+'_'+opb][insert]:
            diags_cc.add('NJN-pi0i')
        t_add=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)+np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0)
        res+=np.conj(t_add[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    t=np.transpose([[get3pt(opa,opb,insert,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

def remove_pi0VEV(Ctij,pi0VEV,opas,opbs):
    if not flags['remove_pi0VEV']:
        return Ctij
    
    factor=yu.coeff['g;0,0,1;G1;N1pi0,a;l1;N,pi'][0][1]
    
    ka=None # index for N(n)pi0(0)
    for i,op in enumerate(opas):
        if yu.getNpar(op)==1:
            qa=i # index for N(n)
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            ka=i
            factor_a = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
    kb=None
    for i,op in enumerate(opbs):
        if yu.getNpar(op)==1:
            qb=i
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            kb=i
            factor_b = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
            
    res=Ctij.copy()
    if ka is not None:
        res[:,ka,:]-=factor_a*Ctij[:,qa,:]
    if kb is not None:
        res[:,:,kb]-=np.conj(factor_b)*Ctij[:,:,qb]
    if ka is not None and kb is not None:
        res[:,ka,kb]+=factor_a*np.conj(factor_b)*Ctij[:,qa,qb]
    
    # print(factor_a,factor_b)
    # print(pi0VEV)
        
    return res

In [ ]:
updateEns(ensembles[1])
ens=ensembles[1]

In [ ]:
t0=1

fig, axs = yu.getFigAxs(1,2,sharex=True,sharey=True)
# yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors'])
yu.addColHeader(axs,[r'$t_0=0.1\,$fm'.format(t0),r'$t-t_0=0.5\,$fm'])

pt='0,0,0'; ofs=[('a','p'),('N1pi1,a','12')]
ops=yu.getops(pt,'l1',ofs)
dat=[get2ptMat(ops),data['VEV']['pi0f']]

eV_func=np.abs
def func(dat):
    t=yu.meanDeep(dat)
    t2pt,tVEVpi0f=t
    t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
    (eVals,eVecs)=yu.GEVP(t2pt,t0)
    eVecs=yu.renormalize_eVecs(eVecs)
    t=eVecs[:,0,:].T
    t=t/t[0,:]
    return eV_func(t)
mean,err,cov=yu.jackknife(dat,func)

xUnit=lat.a
ts=np.array([t for t in range(t0+1,20)])
tmean=mean[1][ts]; terr=err[1][ts]
axs[0,0].errorbar(ts*xUnit,tmean,terr,color='b',mfc='white')
axs[0,0].set_xlabel(r'$t$ [fm]')
axs[0,0].set_ylabel(r'$v_{N,N\pi}$')

dt=5
def func(dat):
    t=yu.meanDeep(dat)
    t2pt,tVEVpi0f=t
    t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
    (eVals,eVecs)=yu.GEVP(t2pt,-dt)
    eVecs=yu.renormalize_eVecs(eVecs)
    t=eVecs[:,0,:].T
    t=t/t[0,:]
    return eV_func(t)
mean,err,cov=yu.jackknife(dat,func)

xUnit=lat.a
ts=np.array([t for t in range(dt+1,17)])
tmean=mean[1][ts]; terr=err[1][ts]
axs[0,1].errorbar(ts*xUnit,tmean,terr,color='b',mfc='white')
ind=7
axs[0,1].errorbar(ts[ind]*xUnit,tmean[ind],terr[ind],color='r')
axs[0,1].set_xlabel(r'$t$ [fm]')
None

In [ ]:


fig, axs = yu.getFigAxs(1,1,sharex=True,sharey=True)
# yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors'])
# yu.addColHeader(axs,[r'$t_0=0.1\,$fm'.format(t0),r'$t-t_0=0.5\,$fm'])

for t0,color,shift in zip([2,7,12],['r','g','b'],[0,0.1,0.2]):
    pt='0,0,0'; ofs=[('a','p'),('N1pi1,a','12')]
    ops=yu.getops(pt,'l1',ofs)
    dat=[get2ptMat(ops),data['VEV']['pi0f']]

    eV_func=np.abs
    def func(dat):
        t=yu.meanDeep(dat)
        t2pt,tVEVpi0f=t
        t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
        (eVals,eVecs)=yu.GEVP(t2pt,t0)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))  # if tRef>=0 else np.log(eVals)/tRef
        return En.T
    mean,err,cov=yu.jackknife(dat,func)

    xUnit=lat.a; yUnit=lat.aInv
    ts=np.array([t for t in range(t0+1,20)])
    tmean=mean[0][ts]; terr=err[0][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color=color,label='1'+str(t0))
    ts=np.array([t for t in range(t0+1,15)])
    tmean=mean[1][ts]; terr=err[1][ts]
    axs[0,0].errorbar(ts*xUnit+shift*xUnit,tmean*yUnit,terr*yUnit,color=color,fmt='d',label='2'+str(t0))
# axs[0,0].set_xlim([0,2.5])
axs[0,0].set_xlabel(r'$t$ [fm]')
axs[0,0].set_ylabel(r'$E_{eff}$ [MeV]')
    
handles, _ = axs[0,0].get_legend_handles_labels()
axs[0,0].legend([handles[idx] for idx in [0,1,0,2,4]],[r'$E_N$',r'$E_{N\pi}$',r'$t_0=0.2\,$fm',r'$t_0=0.7\,$fm',r'$t_0=1.2\,$fm'],fontsize='large')

plt.tight_layout()
plt.savefig('fig/Eeff.pdf')

In [ ]:
tfs=[10,12,14]
FF2simone={
    'GA_00_zz':'GA_0',
    'GA_11_tz':'GA_0',
    'GA_11_zz':'GA_0',
    'GA_11_xx':'GA_0',
    'GA_01':'GA_1',
    'GPb_01_tz':'GPb_1',
    'GPb_01_zz':'GPb_1',
    'G5b_01':'G5b_1',
    'rPCAC_1_Ptz':'rPCAC_1',
    'rPCAC_1_Pzz':'rPCAC_1',
}

simone={
    'GA_0':[1.245,0.031],
    'GA_1':[1.129,0.027],
    'GPb_1':[48.9,2.8],
    'G5b_1':[12.4,0.5],
    'rPCAC_1':[1.06,0.05],
}


ylimDic={
    'GS+_00':[6,22],'GA_00_zz':[0.5,1.6],
    'J5_11':[-2.5,2.5],'GA_11_tz':[0.5,1.6],'GA_11_zz':[0.5,1.6],'GA_11_xx':[0.5,1.6],
    'G5b_01':[0,18],'GA_01':[1,1.2],'GPb_01_tz':[10,60],'GPb_01_zz':[10,60],
    'rPCAC_1_Ptz':[0.2,1.5],'rPCAC_1_Pzz':[0.2,1.5],
}

mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
t_Fpi=92.9/lat.aInv
t_aQ2=(qk**2-(EN-mN)**2)
t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P

t_A2pNN_0=mN/t_Fpi
t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
t_52pNN_1=(lat.amu/lat.ZP[0])*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1

t_GAzz_00=(-1)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA[0]; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
t_GAtz_11=(-1)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA[0]; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
t_GAzz_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2+qk**2)) * lat.ZA[0]; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
t_GAxx_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2)) * lat.ZA[0]; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0

t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA[0]
t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS[0]; t_G5b_01=t_G5_01 * t_52pNN_1
t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA[0]; t_GP_01_tz_A=-(2*mN)/(EN-mN)
t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA[0]; t_GP_01_zz_A=-(2*mN)/(EN-mN)

t_PCAC_G5=(lat.amu/lat.ZP[0])/mN
t_PCAC_GP=t_aQ2/(4*mN**2)

In [ ]:
fig, axs = yu.getFigAxs(2,3,sharex='col',sharey='col')
yu.addRowHeader(axs,['no J-loop','all diags'],fontsize=30)
yu.addColHeader(axs,[r'$G_A$',r'$\bar{G}_P$',r'$m_q\bar{G}_5$'],fontsize=30)

for irow,str_diag in enumerate(['loopless','all']):
    for icol,FF in enumerate(['GA_01','GPb_01_zz','G5b_01']):
        ren={
            'GS+_00':lat.ZP,'GA_00_zz':lat.ZA,'J5_11':lat.ZS,'GA_11_tz':lat.ZA,'GA_11_zz':lat.ZA,'GA_11_xx':lat.ZA,
            'G5b_01':lat.ZSbyZP,'GA_01':lat.ZA,'GPb_01_tz':lat.ZA,'GPb_01_zz':lat.ZA,'rPCAC_1_Ptz':(1,1e-5),'rPCAC_1_Pzz':(1,1e-5)
        }[FF]
        def errRen(mean,err,ren):
            tmean=np.append(mean,1); terr=np.append(err,ren[1]/ren[0]); tcov=np.diag(terr**2)
            if FF in ['J5_11']:
                tcov+=1e-8
            tmean,tcov=yu.propagateError(lambda x:x[:-1]/x[-1],tmean,tcov)
            return tmean,np.sqrt(np.diag(tcov))
        if FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,icol].fill_between([(1-tfs[-1]//2-0.2)*xUnit,(tfs[-1]//2+0.2)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+1].fill_between([(tfs[0]-0.5)*xUnit,(tfs[-1]+0.5)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+2].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+3].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        for case in ['noGEVP']:
            basekey=('0,0,0',tuple([('a','p'),('N1pi1,a','12')]),'0,0,1',tuple([('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'')
            mean,err,_=yu.sl((basekey,ens,str_diag,case,1))
            
            color={'noGEVP':'b','GEVP':'r'}[case]
            mfc={'noGEVP':'white','GEVP':None}[case]
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                tMean,tErr=errRen(tMean,tErr,ren)
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color=color,fmt=['s','d','o'][i_tf],mfc=mfc)  
            axs[irow,icol].set_ylim(ylimDic[FF])
            if irow==1:
                axs[irow,icol].set_xlabel(r'$t_{sink}-t_{ins}/2$ [fm]')    
            

In [ ]:
fig, axs = yu.getFigAxs(2,3,sharex='col',sharey='col')
yu.addRowHeader(axs,['no J-loop','all diags'],fontsize=30)
yu.addColHeader(axs,[r'$G_A$',r'$\bar{G}_P$',r'$m_q\bar{G}_5$'],fontsize=30)

for irow,str_diag in enumerate(['loopless','all']):
    for icol,FF in enumerate(['GA_01','GPb_01_zz','G5b_01']):
        ren={
            'GS+_00':lat.ZP,'GA_00_zz':lat.ZA,'J5_11':lat.ZS,'GA_11_tz':lat.ZA,'GA_11_zz':lat.ZA,'GA_11_xx':lat.ZA,
            'G5b_01':lat.ZSbyZP,'GA_01':lat.ZA,'GPb_01_tz':lat.ZA,'GPb_01_zz':lat.ZA,'rPCAC_1_Ptz':(1,1e-5),'rPCAC_1_Pzz':(1,1e-5)
        }[FF]
        def errRen(mean,err,ren):
            tmean=np.append(mean,1); terr=np.append(err,ren[1]/ren[0]); tcov=np.diag(terr**2)
            if FF in ['J5_11']:
                tcov+=1e-8
            tmean,tcov=yu.propagateError(lambda x:x[:-1]/x[-1],tmean,tcov)
            return tmean,np.sqrt(np.diag(tcov))
        if FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,icol].fill_between([(1-tfs[-1]//2-0.2)*xUnit,(tfs[-1]//2+0.2)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+1].fill_between([(tfs[0]-0.5)*xUnit,(tfs[-1]+0.5)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+2].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+3].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        for case in ['noGEVP','GEVP']:
            basekey=('0,0,0',tuple([('a','p'),('N1pi1,a','12')]),'0,0,1',tuple([('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'')
            mean,err,_=yu.sl((basekey,ens,str_diag,case,1))
            
            color={'noGEVP':'b','GEVP':'r'}[case]
            mfc={'noGEVP':'white','GEVP':None}[case]
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                tMean,tErr=errRen(tMean,tErr,ren)
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color=color,fmt=['s','d','o'][i_tf],mfc=mfc)  
        axs[irow,icol].set_ylim(ylimDic[FF])
        if irow==1:
            axs[irow,icol].set_xlabel(r'$t_{sink}-t_{ins}/2$ [fm]')    
            

In [ ]:
fig, axs = yu.getFigAxs(2,3,sharex='col',sharey='col')
yu.addRowHeader(axs,['no J-loop','all diags'],fontsize=30)
yu.addColHeader(axs,[r'$G_A$',r'$\bar{G}_P$',r'$m_q\bar{G}_5$'],fontsize=30)

for irow,str_diag in enumerate(['loopless','all']):
    for icol,FF in enumerate(['GA_01','GPb_01_zz','G5b_01']):
        ren={
            'GS+_00':lat.ZP,'GA_00_zz':lat.ZA,'J5_11':lat.ZS,'GA_11_tz':lat.ZA,'GA_11_zz':lat.ZA,'GA_11_xx':lat.ZA,
            'G5b_01':lat.ZSbyZP,'GA_01':lat.ZA,'GPb_01_tz':lat.ZA,'GPb_01_zz':lat.ZA,'rPCAC_1_Ptz':(1,1e-5),'rPCAC_1_Pzz':(1,1e-5)
        }[FF]
        def errRen(mean,err,ren):
            tmean=np.append(mean,1); terr=np.append(err,ren[1]/ren[0]); tcov=np.diag(terr**2)
            if FF in ['J5_11']:
                tcov+=1e-8
            tmean,tcov=yu.propagateError(lambda x:x[:-1]/x[-1],tmean,tcov)
            return tmean,np.sqrt(np.diag(tcov))
        if FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            # axs[irow,icol].fill_between([(1-tfs[-1]//2-0.2)*xUnit,(tfs[-1]//2+0.2)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+1].fill_between([(tfs[0]-0.5)*xUnit,(tfs[-1]+0.5)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            axs[irow,icol].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+3].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        for case in ['noGEVP','GEVP']:
            basekey=('0,0,0',tuple([('a','p'),('N1pi1,a','12')]),'0,0,1',tuple([('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'')
            
            color={'noGEVP':'b','GEVP':'r'}[case]
            mfc={'noGEVP':'white','GEVP':None}[case]
            
            shift_chi2=(ylimDic[FF][1]-ylimDic[FF][0])/20
            
            FF2ind={'G5b_01':8,'GA_01':11,'GPb_01_tz':14,'GPb_01_zz':14}
            ind=FF2ind[FF]
            fits=[]
            for datmin in range(1,6):
                if FF=='GA_01' and datmin in [1,2] and case=='noGEVP' and str_diag=='loopless':
                    continue
                res=yu.sl(('fitPCAC',ens,str_diag,case,datmin))
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
                tMean=[pars_mean[ind]]; tErr=[pars_err[ind]]
                tMean,tErr=errRen(tMean,tErr,ren)
                axs[irow,icol].errorbar([datmin*xUnit],tMean,tErr,color=color,mfc=mfc)
                axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean-tErr-shift_chi2),color=color)
                axs[irow,icol].set_ylim(ylimDic[FF])  
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            tMean=pars_mean_MA[ind]; tErr=pars_err_MA[ind]
            tMean,tErr=errRen(tMean,tErr,ren)
            axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
            # fits=[]
            # for datmin in range(1,6):
            #     res=yu.sl((basekey,ens,str_diag,case,3,datmin))
            #     if res is None:
            #         continue
            #     pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
            #     fits.append((pars_mean,pars_err,chi2R_mean,Ndof))
            
            #     tMean=[pars_mean[8]]; tErr=[pars_err[8]]
            #     tMean,tErr=errRen(tMean,tErr,ren)
            #     axs[irow,icol].errorbar([datmin*xUnit],tMean,tErr,color=color,mfc=mfc)
            #     axs[irow,icol].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean-tErr-shift_chi2),color=color)

            # pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            # tMean=pars_mean_MA[8]; tErr=pars_err_MA[8]
            # tMean,tErr=errRen(tMean,tErr,ren)
            # axs[irow,icol].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
            
            axs[irow,icol].set_ylim(ylimDic[FF])
            if irow==1:
                axs[irow,icol].set_xlabel(r'$t_{cut}$ [fm]')
            

In [ ]:
fig, axs = yu.getFigAxs(2,2,sharex='col',sharey='col')
yu.addRowHeader(axs,['no J-loop','all diags'],fontsize=30)
# yu.addColHeader(axs,[r'$G_A$',r'$\bar{G}_P$',r'$m_q\bar{G}_5$'],fontsize=30)

for irow,str_diag in enumerate(['loopless','all']):
    for icol,FF in [(0,'rPCAC_1_Pzz')]:
        shift_chi2=(ylimDic[FF][1]-ylimDic[FF][0])/20
        ren={
            'GS+_00':lat.ZP,'GA_00_zz':lat.ZA,'J5_11':lat.ZS,'GA_11_tz':lat.ZA,'GA_11_zz':lat.ZA,'GA_11_xx':lat.ZA,
            'G5b_01':lat.ZSbyZP,'GA_01':lat.ZA,'GPb_01_tz':lat.ZA,'GPb_01_zz':lat.ZA,'rPCAC_1_Ptz':(1,1e-5),'rPCAC_1_Pzz':(1,1e-5)
        }[FF]
        def errRen(mean,err,ren):
            tmean=np.append(mean,1); terr=np.append(err,ren[1]/ren[0]); tcov=np.diag(terr**2)
            if FF in ['J5_11']:
                tcov+=1e-8
            tmean,tcov=yu.propagateError(lambda x:x[:-1]/x[-1],tmean,tcov)
            return tmean,np.sqrt(np.diag(tcov))
        if FF in FF2simone:
            t_mean,t_err=simone[FF2simone[FF]]
            axs[irow,icol].fill_between([(1-tfs[-1]//2-0.2)*xUnit,(tfs[-1]//2+0.2)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+1].fill_between([(tfs[0]-0.5)*xUnit,(tfs[-1]+0.5)*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            axs[irow,icol+1].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
            # axs[irow,icol+3].fill_between([0*xUnit,6*xUnit],t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        for case in ['noGEVP','GEVP']:
            basekey=('0,0,0',tuple([('a','p'),('N1pi1,a','12')]),'0,0,1',tuple([('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'')
            mean,err,_=yu.sl((basekey,ens,str_diag,case,1))
            
            color={'noGEVP':'b','GEVP':'r'}[case]
            mfc={'noGEVP':'white','GEVP':None}[case]
            for i_tf,tf in enumerate(tfs):
                tMean=mean[i_tf];tErr=err[i_tf]
                tMean,tErr=errRen(tMean,tErr,ren)
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color=color,fmt=['s','d','o'][i_tf],mfc=mfc)  
            axs[irow,icol].set_ylim(ylimDic[FF])
            if irow==1:
                axs[irow,icol].set_xlabel(r'$t_{sink}-t_{ins}/2$ [fm]')   
            
            def estimator(x):
                G5b=x[FF2ind['G5b_01']]
                GA=x[FF2ind['GA_01']]
                GPb=x[FF2ind['GPb_01_tz']]
                ZSbyZPt=x[-2]
                ZAt=x[-1]
                rPCAC=(G5b/t_G5b_01*t_G5_01*t_PCAC_G5*ZSbyZPt + GPb/t_P2pNN_1*t_PCAC_GP*ZAt)/(GA*ZAt)
                return np.array([rPCAC])
            def errRen2(mean,cov):
                N=len(mean)
                tmean=np.concatenate([mean,[1,1]])
                tcov=np.zeros((N+2,N+2))
                tcov[:N,:N]=cov
                tcov[N,N]=(lat.ZSbyZP[1]/lat.ZSbyZP[0])**2
                tcov[N+1,N+1]=(lat.ZA[1]/lat.ZA[0])**2
                return yu.propagateError(estimator,tmean,tcov)
            fits=[]
            for datmin in range(1,6):
                res=yu.sl(('fitPCAC',ens,str_diag,case,datmin))
                if res is None:
                    continue
                pars_mean,pars_err,pars_cov,chi2R_mean,chi2R_err,Ndof,pars0=res
                tMean,tCov=errRen2(pars_mean,pars_cov)
                tErr=np.sqrt(np.diag(tCov))
                fits.append((tMean,tErr,chi2R_mean,Ndof))
                axs[irow,icol+1].errorbar([datmin*xUnit],tMean,tErr,color=color,mfc=mfc)
                axs[irow,icol+1].annotate("%0.1f" %chi2R_mean,(datmin*xUnit,tMean-tErr-shift_chi2),color=color)
                axs[irow,icol+1].set_ylim(ylimDic[FF])  
            pars_mean_MA,pars_err_MA,probs=yu.modelAvg(fits)            
            tMean=pars_mean_MA[0]; tErr=pars_err_MA[0]
            axs[irow,icol+1].fill_between(np.array([1,5])*xUnit,tMean-tErr,tMean+tErr,color=color,alpha=0.2)
            if irow==1:
                axs[irow,icol+1].set_xlabel(r'$t_{cut}$ [fm]')   